In [87]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/10/08 13:58
# @Author  : Wang Yujia
# @File    : PT_demo.ipynb

# @Description : 1. 用SA试做一下inference of PT model's 3 params 2. 复现table 1的code见PT_demo_table1.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Preparations
## 1.1 Import

In [101]:
import numpy as np
import pandas as pd
import sympy
from sko.SA import SA
from tqdm.notebook import tqdm
import seaborn as sns

## 1.2 Global Settings

In [134]:
# auction settings from Swoopo
#b = 0.75
#v = 169.99
#d = 0.15

# auction settings from paper's Fig.1 left
b= 0.75
v = 27.99
d = 0.01

# cnt_row = data_i.shape[0]
cnt_row = 5
# cnt_n_2 = data_i['cnt_n_2']
cnt_n_2 = [1,1,2,3,1]

# from paper's Table 5 'All auction'
alpha = 0.025
delta = 0.85
labda = 3.72
table_5_M = [0.025,0.85,3.72]

# max duration from Swoopo
max_T = 831
# paper setting
#T = int((v-b)/d)

# 2. Infer with SA
1. 用SA 求解参数。
2. 定义函数之后，求一个uniq auction下的u和p
3. 然后用p求NLL作为最终的loss

## 2.1 function definition
1. 根据Eq(7)-(9)

In [106]:
def C(t,b):
    return 0.2*t*b

def OMEGA(p,delta):
    return p**delta * ((p**delta + (1-p)**delta)**(-1/delta))

# valuation function
def f(x, alpha):
    return (1-sympy.E**(-alpha*x))/alpha
    # when x < 0, in fact, it shoule be : (-labda)*(1-sympy.E**(alpha*x))/alpha

def f_Equi(t,v,d,b,alpha,labda,delta):
    u = sympy.Symbol('u')

    tmp = v-d*t-C(t-1,b) - b

    func_1 = (labda * f(x=C(t-1, b), alpha=alpha) - labda * OMEGA(u, delta) * f(x=(C(t-1, b) + b), alpha=alpha) + OMEGA(1-u, delta) * f(tmp, alpha))
    func_2 = (-f(x=C(t-1, b), alpha=alpha) + OMEGA(u, delta) * f(x=(C(t-1, b) + b), alpha=alpha) + (1 - OMEGA(u, delta)) * f(-tmp, alpha))

    if(tmp >= 0):
        return sympy.nsolve(func_1,(0,1),solver='bisect', verify=False)
    else:
        return sympy.nsolve(func_2,(0,1),solver='bisect', verify=False)

## 2.2 Loss func
1. 这个函数比较长 没办法，求u也需要用SA给的params的值。
2. 最后返回NLL

In [147]:
def loss_func(params):

    alpha = params[0]
    delta = params[1]
    labda = params[2]

    #alpha = 0.025
    #labda = 3.72
    #delta = 0.85

    # solve for u&p from Equi. condt.
    U_tmp = [0]*(max_T+1)
    U_tmp[0] = 1
    P_tmp = [0]*(max_T+1)   # P is what we want
    P_tmp[0] = 1

    tmp = 1
    for t in tqdm(range(1,max_T+1),desc="solve for U&P"):

        # if((t>1) & (U_tmp[t-1] > 1-1e-5)):
        #     print("> Break when t == {}, cause U_tmp[t-1] > 1-1e-6".format(t))
        #     break
        U_tmp[t] = f_Equi(t,v,d,b,alpha,labda,delta)
        P_tmp[t] = (1- U_tmp[t])*tmp
        tmp = tmp*U_tmp[t]

    # calculate NLL value under this auction setting & PT params
    nll = 0
    if(P_tmp[0]==1):
        P_tmp.pop(0)            # because P_tmp[0]=1
    P_tmp_df = pd.DataFrame(P_tmp,index=np.arange(0,P_tmp.__len__()),columns=['P'],dtype=float)
    for idx in range(0,cnt_row):
        nll += (np.sum(P_tmp_df[0:(P_tmp_df.shape[0]-1)][:].apply(np.log,axis=1)) + np.log(P_tmp_df.iat[-1,0]) )* cnt_n_2[idx]

    return float(-nll)

In [132]:
# for SA testing
def demo_func(x):
    alpha = x[0]
    delta = x[1]
    labda = x[2]
    return alpha ** 2 + (labda - 0.05) ** 2 + delta ** 2

-16.679542725154533

In [170]:
print("> Initilizing SA....... \n")
from sko.SA import SABoltzmann
from sko.SA import SACauchy

#sa_cauchy = SACauchy(func=loss_func, x0=table_5_M, T_max=100000, T_min=1e-5, learn_rate=0.01, L=50, max_stay_counter=50,
#                     lb=[-0.3,0.01,0.01], ub=[0.3, 2, 16])

sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, T_max=100000, T_min=1e-5, learn_rate=0.01, L=50, max_stay_counter=50,
                            lb=[-0.3,0.01,0.01], ub=[0.3, 2, 16])

#sa = SA(func=loss_func, x0=table_5_M, T_max=100000, T_min=1e-5, L=50, max_stay_counter=50,
#        lb=[-0.3,0.01,0.01], ub=[0.3, 2, 16])

# for SA_Fast:
# [-1.  3. 10.] ok

> Initilizing SA....... 



solve for U&P:   0%|          | 0/831 [00:00<?, ?it/s]

cal NLL:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
print("> Now do SA....... \n")
# best_x, best_y = sa.run()
sa_boltzmann.run()
#sa_cauchy.run()

# print('best_x:', best_x, 'best_y', best_y)
print("SA ends \n")

# sns.lineplot(x = np.arange(0,sa.iter_cycle+1),y=np.array(sa.generation_best_Y))

> Now do SA....... 

-------------- 0_th iteration --------------

---------- 0/L ----------

x_new before clipping:  [ 0.20304637  0.32892831 13.77596819]
x_new after clipping:  [ 0.20304637  0.32892831 13.77596819]


solve for U&P:   0%|          | 0/831 [00:00<?, ?it/s]

cal NLL:   0%|          | 0/5 [00:00<?, ?it/s]

y_new - y_current is 83882.2936966126: 
---------- 0/L ----------

---------- 1/L ----------

x_new before clipping:  [0.03839533 1.203996   2.45467404]
x_new after clipping:  [0.03839533 1.203996   2.45467404]


solve for U&P:   0%|          | 0/831 [00:00<?, ?it/s]

...

y_new - y_current is inf: 
---------- 10/L ----------

---------- 11/L ----------

x_new before clipping:  [-0.14224724  0.06504576 -0.13966848]
x_new after clipping:  [-0.14224724  0.06504576  0.01      ]


solve for U&P:   0%|          | 0/831 [00:00<?, ?it/s]

cal NLL:   0%|          | 0/5 [00:00<?, ?it/s]

y_new - y_current is inf: 
---------- 11/L ----------

---------- 12/L ----------

x_new before clipping:  [-0.1355151   0.81840512 -2.30799881]
x_new after clipping:  [-0.1355151   0.81840512  0.01      ]


solve for U&P:   0%|          | 0/831 [00:00<?, ?it/s]

In [ ]:
# alpha_g = -0.1
# def g(x):
#     return (1-np.exp(-alpha_g*x))/alpha_g
#
# x_g = pd.DataFrame(np.arange(0,100),index=(np.arange(0,100)),columns=['x'])
# x_g['y'] = x_g.apply(g,axis=1)
# sns.scatterplot(x = x_g['x'],y=x_g['y'])